In [1]:
import pandas as pd
from jupyter_dash import JupyterDash
import os
import base64

import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

from crud import AnimalShelter

# Create an instance of AnimalShelter
username = "aacuser"
password = "cheese"
shelter = AnimalShelter(username, password)

# Configure the Dash application
app = JupyterDash('SimpleExample')

# Get the absolute path of the logo file
logo_filename = "Grazioso_Salvare_Logo.png"
logo_path = os.path.join(os.getcwd(), logo_filename)

# Define the preferred breeds and training age for each rescue type
rescue_type_data = {
    'Water': {
        'preferred_breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'preferred_sex': 'Intact Female',
        'training_age': (26, 156)
    },
    'Mountain': {
        'preferred_breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'preferred_sex': 'Intact Male',
        'training_age': (26, 156)
    },
    'Disaster': {
        'preferred_breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'preferred_sex': 'Intact Male',
        'training_age': (20, 300)
    }
}

# Define a function to filter the data based on the selected rescue type
def filter_data(rescue_type):
    if rescue_type == 'reset':
        filtered_data = df.copy()
    else:
        preferred_breeds = rescue_type_data[rescue_type]['preferred_breeds']
        preferred_sex = rescue_type_data[rescue_type]['preferred_sex']
        training_age = rescue_type_data[rescue_type]['training_age']

        filtered_data = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(preferred_breeds)) &
            (df['sex_upon_outcome'] == preferred_sex) &
            (df['age_upon_outcome_in_weeks'].between(*training_age))
        ]

    return filtered_data

# Read data from MongoDB collection into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Convert the logo image to base64 encoding
with open(logo_path, 'rb') as f:
    encoded_logo = base64.b64encode(f.read()).decode()

# Define the application layout
app.layout = html.Div([
    html.Div([
        html.Div([
            html.A([
                html.Img(src='data:image/png;base64,{}'.format(encoded_logo), style={'height': '200px'}),
            ], href='https://www.snhu.edu'),
        ], className='branding')
    ], className='top-center'),
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Branden Langhals - SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        ),
    ]),
    html.Div([
        html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_cell={
                    'minWidth': '0px', 'maxWidth': '180px',
                    'whiteSpace': 'normal'
                },
                page_size=20,
                row_selectable='single',
                selected_rows=[0]
            )
        ], className='col s12 m6'),
        html.Div([
            html.Div(
                id='map-id',
                className='col s12',
            ),
            html.Div(
                id='chart-id',
                className='col s12',
            )
        ], className='col s12 m6')
    ], className='row')
])


# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Function to update the geolocation chart based on selected data entry
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    latitude = float(dff.iloc[row, 13])  # Convert latitude to numeric format
    longitude = float(dff.iloc[row, 14])  # Convert longitude to numeric format

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(dff.iloc[row, 10]),  # Update the column index for 'Outcome Type'
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 8])  # Update the column index for 'name'
                ])
            ])
        ])
    ]


# This callback will update the geolocation chart and the secondary chart for the selected data entry
@app.callback(
    [Output('map-id', "children"), Output('chart-id', "children")],
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map_callback(viewData, index):
    filtered_data = pd.DataFrame.from_dict(viewData)
    if filtered_data.empty:
        chart = html.Div("No data available")
    else:
        fig = px.pie(filtered_data, names='outcome_type', title='Outcome Type Distribution')
        chart = dcc.Graph(figure=fig)

    return update_map(viewData, index), chart


# This callback will filter the data based on the selected rescue type
@app.callback(
    Output('datatable-id', "data"),
    [Input('rescue-type-filter', 'value')]
)
def update_data(rescue_type):
    filtered_data = filter_data(rescue_type)
    return filtered_data.to_dict('records')


# Run the Dash application
if __name__ == '__main__':
    app.run_server(port=8080)


Dash is running on http://127.0.0.1:8080/

Dash app running on http://127.0.0.1:8080/
